In [35]:
import pandas as pd
import csv

import tolka_resultatlista as tkr

def parse_ISUCalcFS_element(line):
    element = {}
    
    element["order"] = int(line[0])
    line = line[1:]
    element["rest"] = line
    """    print(line)
    
    trick = []
    for ch in line[0]:
        pass
        #if  ch 

    if False:
        
        xpos = 2
        jpos = 3
        trick_section = line[1].split()
        print(trick_section, len(trick_section))
        element["order"] = int(line[0])
        element["trick"] = trick_section[0]
        if len(trick_section) == 1:
            element["base"] = float(line[2])
        elif len(trick_section) == 2:
            if "<" in line[1]: 
                element["base"] = float(line[2].replace(",","."))
                xpos = 3
                jpos = 4
            elif "*" in line[1]:
                element["base"] = float(line[2].replace(",","."))
                xpos = 3
                jpos = 4
            elif "e" in line[1]:
                element["base"] = float(line[2].replace(",","."))
                xpos = 3
                jpos = 4
            elif "!" in line[1]:
                element["base"] = float(line[2].replace(",","."))
            else:
                element["base"] = float(trick_section[1].replace(",","."))

        elif len(trick_section) == 3:
            element["base"] = float(trick_section[2].replace(",","."))
            #if "<" in trick_section[1]:
            #    element["base"]  = float(trick_section[2].replace(",","."))
            #elif trick_section[2] == "e" :
            #    element["trick"] += trick_section[2] 
            #    element["base"]  = float(trick_section[4].replace(",","."))
            #else:
        
    #    if trick_section[1] == "!":
    #        element["trick"] +="!"
        
        if "x" in line[xpos]:
            element["late"] =True
            element["GOE"] = line[2].split()[-1] 
            element["GOE"] = float(element["GOE"].replace(",","."))
        else:
            element["GOE"] = float(line[2].replace(",","."))

        if element["base"] > 0:        
            for judge,score in enumerate(line[jpos:-1]):
                if score == "":
                    break
                element["J"+str(judge+1)] = int(score)
        element["score"] = float(line[-1].replace(",","."))"""
    return element

def tolka_ISUCalcFs_resultat(filnamn):

    with open(filnamn,"r") as fil:
        lines = fil.readlines()

    skaters = []
    skater  = {}
    elements = {}

    lines = iter(lines)

    for line in lines:
        words = line.split(",")
        if "Rank" in words[0]:
            next(lines)
            line = csv.reader( [next(lines)] ) 
            line = list(line)
            skater["placed"],skater["name"],skater["nation"],skater["startpos"],skater["TSS"],skater["TES"],skater["PCS"],skater["deductions"] = line[0]
            continue
        if "CATEGORY" in words[0]:
            skater["category"] = words[0].split(":")[-1].strip()
            skater["part"] = words[1].strip()
            continue
        if "#" in words[0]:
            elements = []
            element  = {}
            for line in lines:
                element_line = next( csv.reader([line]) )
                # Line after last element has lots of spaces
                # tabula makes this into empty strings in the middle 
                # of a row. test for empty by joining them up 
                # and checking if the result is empty
                if  "".join(element_line[3:12]) == "":
                    if element_line[1] != "":
                        skater["base score"] = float(element_line[1].replace(",","."))
                    else:
                        skater["base score"] = float(element_line[2].replace(",","."))

                    skater["elements"] = elements
                    elements = {}
                    break
                element = parse_ISUCalcFS_element(element_line)
                elements.append(element)
                
        if "COMPONENTS" in words[0]:
            components = {}
            line = next(lines)
            component_line = next( csv.reader([line]) )
            # Extra ugly hack to handle "Skating skills", don't want to do better atm
            component, factor = component_line[0][:-4], component_line[0][-4:] 
            components[component] = {
                "scores": [float(score.replace(",",".")) for score in component_line[1:-1] if score != "" ],
                "factor": float(factor.replace(",",".")) }

            for line in lines:
                component_line = next( csv.reader([line]) )
                if "".join(component_line[2:7]) == "":
                    skater["components"] =  components
                    components = {}
                    break
                component, factor = component_line[0].split()
                components[component] = {
                    "scores": [float(score.replace(",",".")) for score in component_line[1:-1] if score != "" ],
                    "factor": float(factor.replace(",",".")) }
        #print(words)
        if "Deductions" in words[0]:
            deductions = []
            """for line in lines:
                words = line.split(",")
                if "JUDGES" in words[0]:
                    break

                deductions.append(words[0])
"""
            skater["deductions"] = deductions
            print(skater)
            skaters.append(skater)
            skater = {}
       
    return skaters


In [36]:
filnamn = "tabula-SkateMalmo2018-korta.csv"
skaters = tolka_ISUCalcFs_resultat(filnamn)

{'elements': [{'rest': ['3Lz+COMBO  6.00', '-2.10', '-3', '-3', '-3', '-3', '-3', '', '', '', '', '', '3.90'], 'order': 1}, {'rest': ['3S  4.40', '-1.63', '-2', '-3', '-2', '-3', '-2', '', '', '', '', '', '2.77'], 'order': 2}, {'rest': ['FCSp3  2.80', '0.50', '1', '1', '0', '1', '1', '', '', '', '', '', '3.30'], 'order': 3}, {'rest': ['2A  3.30', '-0.67', '-1', '-2', '-1', '-1', '-2', '', '', '', '', '', '2.63'], 'order': 4}, {'rest': ['LSp4  2.70', '0.83', '2', '2', '1', '2', '1', '', '', '', '', '', '3.53'], 'order': 5}, {'rest': ['StSq3  3.30', '1.00', '2', '2', '2', '3', '2', '', '', '', '', '', '4.30'], 'order': 6}, {'rest': ['CCoSp4  3.50', '1.00', '2', '2', '2', '2', '2', '', '', '', '', '', '4.50'], 'order': 7}], 'deductions': [], 'PCS': '22.08', 'TSS': '47.01', 'TES': '24.93', 'placed': '1', 'nation': 'SWE', 'base score': 26.0, 'name': 'Matilda ALGOTSSON', 'startpos': '10'}
{'elements': [{'rest': ['StSq2  2.60', '0.83', '2', '2', '1', '2', '1', '', '', '', '', '', '3.43'], 'or

In [6]:
with open(filnamn,"r") as fil:
        lines = fil.readlines()
for line in lines[:100]:
    print(line)

"",,,Starting,Total,Total,Total,Total

Rank,Name,Nation,Number,Segment,Element,Program  Component,Deductions

"",,,,Score,Score,Score (factored),

1,Matilda ALGOTSSON,SWE,10,47.01,24.93,22.08,0.00

"",Executed Base,,,,,,,,,,,,Scores of

"#",GOE Elements Value,,J1,J2,J3,J4,J5,J6,J7,IJn8fo,J9,Ref,Panel

1,3Lz+COMBO  6.00,-2.10,-3,-3,-3,-3,-3,,,,,,3.90

2,3S  4.40,-1.63,-2,-3,-2,-3,-2,,,,,,2.77

3,FCSp3  2.80,0.50,1,1,0,1,1,,,,,,3.30

4,2A  3.30,-0.67,-1,-2,-1,-1,-2,,,,,,2.63

5,LSp4  2.70,0.83,2,2,1,2,1,,,,,,3.53

6,StSq3  3.30,1.00,2,2,2,3,2,,,,,,4.30

7,CCoSp4  3.50,1.00,2,2,2,2,2,,,,,,4.50

"",26.00,,,,,,,,,,,,24.93

"",Program Components,Factor,,,,,,,,,,,

"",Skating Skills,0.80,"6,25","5,75","5,75","5,25","5,75",,,,,,5.75

"",Transitions,0.80,"6,00","5,25","5,50","5,25","5,50",,,,,,5.42

"",Performance,0.80,"6,50","5,50","5,50","5,25","5,25",,,,,,5.42

"",Composition,0.80,"6,50","5,50","5,50","5,50","5,50",,,,,,5.50

"",Interpretation of the Music,0.80,"6,50","5,25","5,75","5,25","5